In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gmplot 
import statistics
from pandas import Series
from datetime import time
from datetime import date
from datetime import datetime as dt

In [2]:
tas = pd.read_csv('./data/tas.csv')
tas=tas.fillna(0)
tas_rt =[]
tas_rs =[]
tas_res=[]
tas_ret=[]
temp_index=[]
temp_index2=[]
temp_rate=[]
temp_rate2=[]
temp = tas.iloc[:,1]
temp2 = tas.iloc[:,3]
for i,v in enumerate(tas['RENT_STATION']):
    r_s = int(float(str(v)))
    if r_s < 145 and r_s>0:
        temp_index.append(i)
        tas_rs.append(r_s)
for i in temp_index:
    temp_rate.append(temp[i])
temp_rate=pd.DataFrame(temp_rate,columns=['RENT_DATE'])

for i,v in enumerate(temp_rate['RENT_DATE']):
    r_d = str(v)[0:10]
    tas_rt.append(int(r_d))
    
for i,v in enumerate(tas['RETURN_STATION']):
    r_s = int(float(str(v)))
    if r_s < 145:
        temp_index2.append(i)
        tas_res.append(r_s)
for i in temp_index2:
    temp_rate2.append(temp2[i])

temp_rate2=pd.DataFrame(temp_rate2,columns=['RETURN_DATE'])

for i,v in enumerate(temp_rate2['RETURN_DATE']):
    r_d = str(v)[0:10]
    tas_ret.append(r_d)    
        
tas_rt = pd.DataFrame(tas_rt,columns=['RENT_DATE'])
tas_rs = pd.DataFrame(tas_rs,columns=['STATION'])
tas_ret = pd.DataFrame(tas_ret,columns=['RETURN_DATE'])
tas_res = pd.DataFrame(tas_res,columns=['STATION'])

rent_tas = pd.merge(tas_rs,tas_rt,how='outer',left_index=True ,right_index=True)
retu_tas = pd.merge(tas_res,tas_ret,how='outer',left_index=True ,right_index=True)
rent_tas = pd.DataFrame(rent_tas,columns=('STATION','RENT_DATE'))
retu_tas = pd.DataFrame(retu_tas,columns=('STATION','RETURN_DATE'))

rent_tas=rent_tas[rent_tas.STATION != 0]

train = rent_tas
train2 = retu_tas
train['RENT_COUNT']= pd.Series(0,index=train.index)


In [3]:
ret_time = retu_tas.iloc[:,1]
ret_stat = retu_tas.iloc[:,0]

In [4]:
rent_time= rent_tas.iloc[:,1]
rent_stat = rent_tas.iloc[:,0]

r_year = []
r_month =[]
r_day = []
r_hour =[]
weekday = []
wet = pd.read_csv('./data/wet.csv')
w_year=[]
w_month =[]
w_day=[]
w_hour=[]

season =[]
wet=wet.fillna(0)

temp = wet.iloc[:,0]
for i,v in enumerate(temp):
    dat= str(v)
    if len(dat)>11:
        w_year.append(int(dat[0:4]))
        w_month.append(int(dat[4:6]))
        w_day.append(int(dat[6:8]))
        w_hour.append(int(dat[8:10]))
    else:
        w_year.append(int(dat[0:4]))
        w_month.append(int(dat[4:6]))
        w_day.append(int(dat[6:8]))
        w_hour.append(int(dat[8:9]))
del wet['TIME']


for i in range(len(w_year)):
    if int(w_month[i])>= 3 and int(w_month[i]) <= 5:
        season.append(0)
    elif int(w_month[i])>=6 and int(w_month[i]) <=8:
        season.append(1)
    elif int(w_month[i])>=9 and int(w_month[i]) <=11:
        season.append(2)
    else:
        season.append(3)
for i,v in enumerate(rent_time):
    temp = str(v)
    r_year.append(int(temp[0:4]))
    r_month.append(int(temp[4:6]))
    r_day.append(int(temp[6:8]))
    r_hour.append(int(temp[8:10]))

for i in range(len(w_year)):
    date_t = date(r_year[i],r_month[i],r_day[i])
    temp=str(date_t.weekday())
    weekday.append(temp)    
    
r_year = pd.DataFrame(r_year,columns=['YEAR'])
r_month =pd.DataFrame(r_month,columns=['MONTH'])
r_day = pd.DataFrame(r_day,columns=['DAY'])
r_hour = pd.DataFrame(r_hour,columns=['HOUR'])   
c_rent = pd.merge(r_year,r_month,how='outer',left_index=True,right_index=True)
c_rent = pd.merge(c_rent,r_day,how='outer',left_index=True,right_index=True)
c_rent = pd.merge(c_rent,r_hour,how='outer',left_index=True,right_index=True)
c_rent = pd.merge(c_rent,rent_tas,how='outer',left_index=True,right_index=True)
del c_rent['RENT_DATE']
c_rent=pd.DataFrame(c_rent,columns=['STATION','YEAR','MONTH','DAY','HOUR','RENT_COUNT'])
a=c_rent.groupby(['STATION','YEAR','MONTH','DAY','HOUR'])['RENT_COUNT'].count().reset_index()
a=pd.DataFrame(a)


In [5]:

station1= pd.read_csv('./data/station.csv')
station1.head(7)

,번호,키오스크번호,구별,명칭,위치,주소,거치대,좌표
0,1,1,유성구,무역전시관입구(택시승강장 앞),엑스포다리 맞은편,유성구 도룡동 3-8,14,"36.374325,127.387462"
1,2,2,유성구,대전컨벤션 센터 앞,둔산대교 맞은편,유성구 도룡동 4-19,20,"36.374472,127.392241"
2,3,3,서구,한밭수목원(정문입구),한밭수목원 내,서구 만년동 396,19,"36.369855,127.388749"
3,4,4,서구,초원아파트104동부근(버스정류장),초원아파트 104동앞 쪽문 육교 버스정류장 앞,서구 만년동 401,12,"36.368192,127.379281"
4,5,5,서구,둔산대공원 입구(버스정류장),한밭수목원에서 평송수련원 가는길 버스정류장 앞,서구 둔산동 1521-10,13,"36.365034,127.389361"
5,6,6,서구,백합4가 앞(농협앞),백합아파트 상가 농협 버스정류장 앞,서구 월평2동 266,12,"36.362304,127.376419"
6,7,7,서구,정부청사 입구(대덕대로),둔산 시외버스터미널 버스정류장 앞,서구 둔산동 920-2,13,"36.361665,127.379699"


In [6]:

weekday = pd.DataFrame(weekday,columns=['WEEKDAY'])
season = pd.DataFrame(season,columns=['SEASON'])
w_year = pd.DataFrame(w_year,columns=['YEAR'])
w_month =pd.DataFrame(w_month,columns=['MONTH'])
w_day = pd.DataFrame(w_day,columns=['DAY'])
w_hour = pd.DataFrame(w_hour,columns=['HOUR'])

total_rent = pd.merge(w_year,w_month,how='outer',left_index=True,right_index=True)
total_rent = pd.merge(total_rent,w_day,how='outer',left_index=True,right_index=True)
total_rent = pd.merge(total_rent,w_hour,how='outer',left_index=True,right_index=True)
total_rent2 = pd.merge(weekday,season,how='outer',left_index=True,right_index=True)
total_rent2 = pd.merge(total_rent2,wet,how='outer',left_index=True,right_index=True)
station=[]
te = total_rent
te2 = total_rent2
for i in range(144):
    total_rent=pd.concat([total_rent,te],axis=0,ignore_index=True)
    total_rent2=pd.concat([total_rent2,te2],axis=0,ignore_index=True)
    for j in range(26280):
        station.append(i+1)

station=pd.DataFrame(station,columns=["STATION"])

total_rent=pd.merge(station,total_rent,how="inner",left_index=True,right_index=True)
total_rent['RENT_COUNT2']= pd.Series(0,index=total_rent.index)


In [7]:
result = pd.merge(total_rent,a,how="outer")

In [8]:
result= result.fillna(0)
a = result.iloc[:,-1]
b = result.iloc[:,-2]
ren=a+b
ren=pd.DataFrame(ren,columns=["RENT_COUNT"])
del result['RENT_COUNT']
del result['RENT_COUNT2']
result = pd.merge(result,ren,how="outer",left_index=True,right_index=True)

In [9]:
a2 = result.iloc[:,-1]
b2 = result.iloc[:,-2]
ren2=a+b
ren2=pd.DataFrame(ren2,columns=["RENT_COUNT"])

In [10]:
result=result.loc[0:3784319]
total_rent2=total_rent2.loc[0:3784319]
result=pd.merge(result,total_rent2,how="outer",left_index=True,right_index=True)

result

,STATION,YEAR,MONTH,DAY,HOUR,RENT_COUNT,WEEKDAY,SEASON,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,LOCALPRESSURE,SNOWFALL
0,1,2013,1,1,0,0.0,1,3,8.8,0.0,0.1,90.0,1013.3,8.8
1,1,2013,1,1,1,0.0,1,3,8.5,0.0,0.9,90.0,1013.2,8.8
2,1,2013,1,1,2,0.0,1,3,8.5,0.0,1.0,89.0,1012.6,8.8
3,1,2013,1,1,3,0.0,1,3,9.0,0.0,0.7,91.0,1012.5,8.8
4,1,2013,1,1,4,0.0,1,3,9.1,0.0,0.6,92.0,1011.4,8.8
5,1,2013,1,1,5,0.0,1,3,9.4,0.0,0.5,92.0,1010.4,8.8
6,1,2013,1,1,6,0.0,1,3,9.0,0.0,1.4,93.0,1009.7,8.8
7,1,2013,1,1,7,0.0,1,3,8.3,0.0,0.2,94.0,1009.1,9.6
8,1,2013,1,1,8,0.0,1,3,7.3,0.0,1.7,96.0,1008.9,10.5
9,1,2013,1,1,9,0.0,1,3,6.4,2.2,0.9,96.0,1009.4,12.8


In [11]:
train2['RETURN_COUNT']= pd.Series(0,index=train2.index)

In [18]:
retu_tas=retu_tas[retu_tas.STATION != 0]
retu_tas=retu_tas[retu_tas.RETURN_DATE != 0.0]
retu_tas

,STATION,RETURN_DATE
0,34,2013010106
2,10,2013010106
3,105,2013010110
4,4,2013010112
5,105,2013010111
6,91,2013010112
7,30,2013010113
8,1,2013010113
9,2,2013010115
10,2,2013010115


In [27]:
re_year = []
re_month =[]
re_day = []
re_hour =[]

for i,v in enumerate(ret_time):
    temp = str(v)
    if temp != '0.0':
        re_year.append(int(temp[0:4]))
        re_month.append(int(temp[4:6]))
        re_day.append(int(temp[6:8]))
        re_hour.append(int(temp[8:10]))
        
re_year = pd.DataFrame(re_year,columns=['YEAR'])
re_month =pd.DataFrame(re_month,columns=['MONTH'])
re_day = pd.DataFrame(re_day,columns=['DAY'])
re_hour = pd.DataFrame(re_hour,columns=['HOUR'])  
c_retu = pd.merge(re_year,re_month,how='outer',left_index=True,right_index=True)
c_retu = pd.merge(c_retu,re_day,how='outer',left_index=True,right_index=True)
c_retu = pd.merge(c_retu,re_hour,how='outer',left_index=True,right_index=True)
c_retu = pd.merge(c_retu,retu_tas,how='outer',left_index=True,right_index=True)
del c_retu['RETURN_DATE']
c_retu=pd.DataFrame(c_retu,columns=['STATION','YEAR','MONTH','DAY','HOUR','RETURN_COUNT'])
c_retu=c_retu.fillna(0)
c_retu=c_retu[c_retu.STATION!=0.0]
re_a=c_retu.groupby(['STATION','YEAR','MONTH','DAY','HOUR'])['RETURN_COUNT'].count().reset_index()
re_a=pd.DataFrame(re_a)
re_a

,STATION,YEAR,MONTH,DAY,HOUR,RETURN_COUNT
0,1.0,2013.0,1.0,1.0,13.0,1
1,1.0,2013.0,1.0,2.0,7.0,1
2,1.0,2013.0,1.0,2.0,17.0,1
3,1.0,2013.0,1.0,2.0,23.0,2
4,1.0,2013.0,1.0,3.0,8.0,1
5,1.0,2013.0,1.0,3.0,13.0,1
6,1.0,2013.0,1.0,4.0,23.0,1
7,1.0,2013.0,1.0,5.0,8.0,1
8,1.0,2013.0,1.0,5.0,14.0,1
9,1.0,2013.0,1.0,5.0,15.0,1


In [29]:
result2 = pd.merge(total_rent,re_a,how="outer")
result2= result2.fillna(0)
a = result2.iloc[:,-1]
b = result2.iloc[:,-2]
ren=a+b
ren=pd.DataFrame(ren,columns=["RETURN_COUNT"])
del result2['RENT_COUNT2']
del result2['RETURN_COUNT']
result2 = pd.merge(result2,ren,how="outer",left_index=True,right_index=True)
result2

,STATION,YEAR,MONTH,DAY,HOUR,RETURN_COUNT
0,1,2013,1,1,0,0.0
1,1,2013,1,1,1,0.0
2,1,2013,1,1,2,0.0
3,1,2013,1,1,3,0.0
4,1,2013,1,1,4,0.0
5,1,2013,1,1,5,0.0
6,1,2013,1,1,6,0.0
7,1,2013,1,1,7,0.0
8,1,2013,1,1,8,0.0
9,1,2013,1,1,9,0.0


In [30]:
result2=result2.loc[0:3784319]
result2=pd.merge(result2,total_rent2,how="outer",left_index=True,right_index=True)
result2=pd.merge(result2,ren2,how="outer",left_index=True,right_index=True)


,STATION,YEAR,MONTH,DAY,HOUR,RETURN_COUNT,WEEKDAY,SEASON,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,LOCALPRESSURE,SNOWFALL,RENT_COUNT
0,1.0,2013.0,1.0,1.0,0.0,0.0,1,3.0,8.8,0.0,0.1,90.0,1013.3,8.8,0.0
1,1.0,2013.0,1.0,1.0,1.0,0.0,1,3.0,8.5,0.0,0.9,90.0,1013.2,8.8,0.0
2,1.0,2013.0,1.0,1.0,2.0,0.0,1,3.0,8.5,0.0,1.0,89.0,1012.6,8.8,0.0
3,1.0,2013.0,1.0,1.0,3.0,0.0,1,3.0,9.0,0.0,0.7,91.0,1012.5,8.8,0.0
4,1.0,2013.0,1.0,1.0,4.0,0.0,1,3.0,9.1,0.0,0.6,92.0,1011.4,8.8,0.0
5,1.0,2013.0,1.0,1.0,5.0,0.0,1,3.0,9.4,0.0,0.5,92.0,1010.4,8.8,0.0
6,1.0,2013.0,1.0,1.0,6.0,0.0,1,3.0,9.0,0.0,1.4,93.0,1009.7,8.8,0.0
7,1.0,2013.0,1.0,1.0,7.0,0.0,1,3.0,8.3,0.0,0.2,94.0,1009.1,9.6,0.0
8,1.0,2013.0,1.0,1.0,8.0,0.0,1,3.0,7.3,0.0,1.7,96.0,1008.9,10.5,0.0
9,1.0,2013.0,1.0,1.0,9.0,0.0,1,3.0,6.4,2.2,0.9,96.0,1009.4,12.8,0.0


In [31]:
result2=result2.loc[0:3784319]
result2

,STATION,YEAR,MONTH,DAY,HOUR,RETURN_COUNT,WEEKDAY,SEASON,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,LOCALPRESSURE,SNOWFALL,RENT_COUNT
0,1.0,2013.0,1.0,1.0,0.0,0.0,1,3.0,8.8,0.0,0.1,90.0,1013.3,8.8,0.0
1,1.0,2013.0,1.0,1.0,1.0,0.0,1,3.0,8.5,0.0,0.9,90.0,1013.2,8.8,0.0
2,1.0,2013.0,1.0,1.0,2.0,0.0,1,3.0,8.5,0.0,1.0,89.0,1012.6,8.8,0.0
3,1.0,2013.0,1.0,1.0,3.0,0.0,1,3.0,9.0,0.0,0.7,91.0,1012.5,8.8,0.0
4,1.0,2013.0,1.0,1.0,4.0,0.0,1,3.0,9.1,0.0,0.6,92.0,1011.4,8.8,0.0
5,1.0,2013.0,1.0,1.0,5.0,0.0,1,3.0,9.4,0.0,0.5,92.0,1010.4,8.8,0.0
6,1.0,2013.0,1.0,1.0,6.0,0.0,1,3.0,9.0,0.0,1.4,93.0,1009.7,8.8,0.0
7,1.0,2013.0,1.0,1.0,7.0,0.0,1,3.0,8.3,0.0,0.2,94.0,1009.1,9.6,0.0
8,1.0,2013.0,1.0,1.0,8.0,0.0,1,3.0,7.3,0.0,1.7,96.0,1008.9,10.5,0.0
9,1.0,2013.0,1.0,1.0,9.0,0.0,1,3.0,6.4,2.2,0.9,96.0,1009.4,12.8,0.0


In [32]:
result2.to_csv("./data/final_data.csv",mode='w')

,RENT_COUNT,RETURN_COUNT,STATION,YEAR,MONTH,DAY,HOUR,SEASON,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,LOCALPRESSURE,SNOWFALL
0,0.0,0.0,180534473.0,4.492032e+09,14305226.0,34888413.0,21124159.0,3561435.0,2.856739e+07,379221.0,3.281986e+06,168499471.7,2.250222e+09,735229.1
1,0.0,1.0,19311035.0,5.691490e+08,1882221.0,4455798.0,3571052.0,412967.0,4.278345e+06,18213.6,4.603404e+05,19413316.2,2.848922e+08,22086.7
2,0.0,2.0,6847856.0,2.135511e+08,713592.0,1681845.0,1394431.0,148023.0,1.679846e+06,5903.9,1.714940e+05,7237089.8,1.068613e+08,4709.2
3,0.0,3.0,2459059.0,8.321735e+07,280496.0,653873.0,552787.0,53879.0,6.920562e+05,1572.4,6.665340e+04,2811835.3,4.161781e+07,1238.1
4,0.0,4.0,1020549.0,3.613725e+07,122239.0,286454.0,239042.0,22272.0,3.089583e+05,510.4,2.882770e+04,1214787.0,1.806792e+07,285.0
5,0.0,5.0,432155.0,1.588739e+07,54409.0,125287.0,103314.0,9411.0,1.394591e+05,244.5,1.249600e+04,537812.0,7.940648e+06,110.4
6,0.0,6.0,178751.0,6.986951e+06,23915.0,54833.0,44193.0,4007.0,6.161150e+04,84.4,5.592600e+03,237500.1,3.491451e+06,40.8
7,0.0,7.0,74726.0,3.071566e+06,10523.0,24709.0,19165.0,1725.0,2.724720e+04,28.6,2.398000e+03,104266.1,1.534631e+06,46.1
8,0.0,8.0,32220.0,1.369576e+06,4651.0,10921.0,8367.0,778.0,1.207260e+04,45.2,1.049400e+03,46976.0,6.844352e+05,5.5
9,0.0,9.0,15388.0,6.485800e+05,2269.0,5646.0,3690.0,382.0,5.797300e+03,0.3,5.033000e+02,21988.0,3.241025e+05,0.0
